In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from openai import OpenAI
from dotenv import load_dotenv, dotenv_values
load_dotenv()

client = OpenAI()

In [5]:
# URL of the page you want to scrape
base_url = "https://help.relativity.com/RelativityOne/Content/Relativity/aiR_for_Review/aiR_for_Review.htm"
llm_input = ''

# Send a GET request to the webpage
response = requests.get(base_url)

# If the GET request is successful, the status code will be 200
if response.status_code == 200:
    # Get the content of the response
    content = response.content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(content, 'html.parser')

    # Find the sub-menu element
    sub_menu = soup.find(class_='guideTOC')

    # Find all anchor tags within the sub-menu and extract href attributes
    links = [urljoin(base_url, a['href']) for a in sub_menu.find_all('a', href=True)]

    # Loop through the links
    for link in links:
        # Send a GET request to the child page
        response = requests.get(link)

        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
            # Get the content of the response
            content = response.content

            # Create a BeautifulSoup object and specify the parser
            soup = BeautifulSoup(content, 'html.parser')

            # Find the specified div tags
            paragraphs = soup.find_all('div', id="mc-main-content")

            # Loop through the paragraphs, get the text of each one, and print it
            for paragraph in paragraphs:
                llm_input += paragraph.get_text()
        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

print(llm_input)


Running aiR for Review
aiR for Review uses generative AI to simulate the actions of a human reviewer, finding and describing relevant documents using the review instructions that you provide. These analyses can be customized to search for relevance, key documents, or specific issues as needed.
The instructions you give aiR for Review are called Prompt Criteria. For best results, we recommend analyzing a small set of documents, tweaking the Prompt Criteria as needed, then finally analyzing a larger set of documents. This lets you see immediately how aiR's coding compares to a human reviewer's coding and adjust the prompts accordingly.
Note: aiR for Review is currently in limited release. For information about the general release, contact your account representative.
See these related pages:

aiR for Review

Monitoring aiR for Review jobs

aiR for Review results

aiR for Review security permissions


See these additional resources:

Workflows for Applying aiR for Review

aiR for Review 

In [7]:
#Pass the documentation text to GPT-3.5 Turbo and ask a question.

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"You are a teacher helping me understand the following documentation on the aiR for Review product: {llm_input}"},
    {"role": "user", "content": f"What permissions are needed to run aiR for Review?"}
  ]
)

print(completion.choices[0].message.content)

To run aiR for Review, you need the following permissions:

1. **aiR for Review Profile - View, Edit, Add**: This permission allows you to interact with aiR for Review, including setting up, editing, and adding new profiles.

2. **Mass Operations - aiR for Review**: This permission enables you to run mass operations related to aiR for Review, such as analyzing documents, setting up prompt criteria, and submitting jobs for analysis.

Make sure you also belong to at least one user group other than the Workspace Admin Group for the system to recognize and authorize you to run aiR for Review.
